<a href="https://colab.research.google.com/github/EML-Labs/Dataset/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 102.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.


In [5]:
import wfdb
import numpy as np
import os
import glob

data = "/content/drive/MyDrive/Datasets/1.0.0"
save_dir = "/content/drive/MyDrive/Datasets/1.0.0_pro"

def segmentAndLabel(record_path, record_name, window_sec=10, overlap=0.0, save_dir='segments'):
    record = wfdb.rdrecord(os.path.join(record_path, record_name))
    signal = record.p_signal
    fs = record.fs
    window_size = int(window_sec * fs)
    step_size = int(window_size * (1 - overlap))

    if record_name.startswith('n'):
        label = 'sn'
    elif record_name.endswith('c'):
        label = 'af'
    else:
        label = 'pre-af'

    os.makedirs(save_dir, exist_ok=True)

    segment_id = 0
    for start in range(0, len(signal) - window_size + 1, step_size):
        segment = signal[start:start + window_size]
        npy_filename = f"{record_name}_seg{segment_id:03d}_{label}.npy"
        np.save(os.path.join(save_dir, npy_filename), segment)
        segment_id += 1

    print(f"Saved {segment_id} segments for {record_name} with label '{label}'")

hea_files = sorted(glob.glob(os.path.join(data, '*.hea')))
record_names = [
    os.path.splitext(os.path.basename(f))[0]
    for f in hea_files
    if not os.path.basename(f).startswith('t')
]

for record_name in record_names:
    segmentAndLabel(record_path=data,
                      record_name=record_name,
                      window_sec=10,
                      overlap=0.0,
                      save_dir=save_dir)

Saved 180 segments for n01 with label 'sn'
Saved 30 segments for n01c with label 'sn'
Saved 180 segments for n02 with label 'sn'
Saved 30 segments for n02c with label 'sn'
Saved 180 segments for n03 with label 'sn'
Saved 30 segments for n03c with label 'sn'
Saved 180 segments for n04 with label 'sn'
Saved 30 segments for n04c with label 'sn'
Saved 180 segments for n05 with label 'sn'
Saved 30 segments for n05c with label 'sn'
Saved 180 segments for n06 with label 'sn'
Saved 30 segments for n06c with label 'sn'
Saved 180 segments for n07 with label 'sn'
Saved 30 segments for n07c with label 'sn'
Saved 180 segments for n08 with label 'sn'
Saved 30 segments for n08c with label 'sn'


KeyboardInterrupt: 

In [6]:
from google.colab import drive
import os
import wfdb
import numpy as np
import glob
import csv

# 2. Paths
raw_data_dir = "/content/drive/MyDrive/Datasets/1.0.0"
segment_save_dir = "/content/drive/MyDrive/Datasets/1.0.0_processed"
os.makedirs(segment_save_dir, exist_ok=True)

# 3. Initialize label file
label_file = os.path.join(segment_save_dir, 'labels.csv')
csv_rows = [['filename', 'label']]

# 4. Segmentation + Labeling Function
def segment_and_label(record_path, record_name, label, window_sec=30, overlap=0.0, save_dir='segments'):
    try:
        record = wfdb.rdrecord(os.path.join(record_path, record_name))
        signal = record.p_signal
        fs = record.fs
        window_size = int(window_sec * fs)
        step_size = int(window_size * (1 - overlap))

        os.makedirs(save_dir, exist_ok=True)

        segment_id = 0
        for start in range(0, len(signal) - window_size + 1, step_size):
            segment = signal[start:start + window_size]
            fname = f"{record_name}_seg{segment_id:03d}.npy"
            np.save(os.path.join(save_dir, fname), segment)
            csv_rows.append([fname, label])
            segment_id += 1

        print(f"Saved {segment_id} segments for {record_name} with label {label}")
    except Exception as e:
        print(f"Failed to process {record_name}: {e}")

# 5. Get list of records starting with 'p', excluding those ending with 'c'
hea_files = sorted(glob.glob(os.path.join(raw_data_dir, 'p*.hea')))
record_names = [
    os.path.splitext(os.path.basename(f))[0]
    for f in hea_files
    if not os.path.basename(f).endswith('c')
]

# 6. Process records
for record_name in record_names:
    try:
        rec_num = int(record_name[1:])  # Get number after 'p'
        label = 1 if rec_num % 2 == 0 else 0  # Even = pre-af (1), Odd = non-af (0)
        segment_and_label(record_path=raw_data_dir,
                          record_name=record_name,
                          label=label,
                          window_sec=30,
                          overlap=0.0,
                          save_dir=segment_save_dir)
    except ValueError:
        print(f"Skipping malformed record name: {record_name}")

# 7. Write labels.csv
with open(label_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(csv_rows)

print(f"Saved label file to: {label_file}")



Saved 60 segments for p01 with label 0
Skipping malformed record name: p01c
Saved 60 segments for p02 with label 1
Skipping malformed record name: p02c
Saved 60 segments for p03 with label 0
Skipping malformed record name: p03c
Saved 60 segments for p04 with label 1
Skipping malformed record name: p04c
Saved 60 segments for p05 with label 0
Skipping malformed record name: p05c
Saved 60 segments for p06 with label 1
Skipping malformed record name: p06c
Saved 60 segments for p07 with label 0
Skipping malformed record name: p07c
Saved 60 segments for p08 with label 1
Skipping malformed record name: p08c
Saved 60 segments for p09 with label 0
Skipping malformed record name: p09c
Saved 60 segments for p10 with label 1
Skipping malformed record name: p10c
Saved 60 segments for p11 with label 0
Skipping malformed record name: p11c
Saved 60 segments for p12 with label 1
Skipping malformed record name: p12c
Saved 60 segments for p13 with label 0
Skipping malformed record name: p13c
Saved 60 seg